In [1]:
# importing necessary packages
import numpy as np
import pandas as pd
import math
from scipy import sparse

In [2]:
# Global variables
tol_var = 10**-10

In [19]:
# Function for power iteration taking matrices as input
def power_iteration(s_matrix):
    one_vec = np.ones((tot_num_of_nodes,1))
    counter = 0
    # initialise the r_0
    r_var = 1/ float(tot_num_of_nodes) * one_vec
    while True:
        temp_1 = (1-0.8)/tot_num_of_nodes
        temp_2 = 0.8 * s_matrix.dot(r_var)
        r_new_var = temp_1 * one_vec + temp_2
        L_1_Norm = np.linalg.norm(abs(r_new_var - r_var), ord=1)
        if L_1_Norm < tol_var:
            break
        
        r_var = r_new_var
        counter = counter + 1
    
    print('For tolerance - ',tol_var,'Total number of iterations for power iteration it took is :- ', counter)
    print('\n')
    return r_var

In [20]:
def power_extrapolation(T_matrix, tot_num_of_nodes):
    d_val = 8
    dict_page_ranks = {}
    one_vec = np.ones((tot_num_of_nodes,1))
    initial_page_rank = 1 / tot_num_of_nodes * one_vec
    page_rank_itr_list = []
    page_rank_itr_list.append(initial_page_rank)
    ite = 0
    while True:
        ite = ite + 1
        temp = (1 - 0.8) / tot_num_of_nodes
        page_rank_next = temp * one_vec + 0.8 * T_matrix.dot(initial_page_rank)
        if ite == d_val + 2:
            temp_1 = (0.8**d_val)*page_rank_itr_list[ite - d_val]
            temp_2 = (1 - 0.8**d_val)
            page_rank_next = (page_rank_next - temp_1) / temp_2
        
        error = np.linalg.norm(abs(page_rank_next - initial_page_rank), ord=1)
        if error < tol_var:
            page_rank_itr_list.append(page_rank_next)
            break
        
        page_rank_itr_list.append(page_rank_next)
        initial_page_rank = page_rank_next
    
    dict_page_ranks[d_val] = page_rank_itr_list
    print('Total number of iterations for power extrapolation it took is :- ', ite)
    print('\n')
    return dict_page_ranks.get(d_val)[-1]

In [6]:
def getTopK(k_top,r):
    # Top k page ranks with values
    k_top_Pages_id = r.T[0].argsort()[-k_top:][::-1]
    k_top_Page_list = []
    for ind in k_top_Pages_id:
        k_top_Page_list.append((r[ind][0], ind+1))
    
    # Printing top k nodes with scores
    print('---------------------Top k node ids with scores------------------------')
    for pr in enumerate(k_top_Page_list):
        (ind, (pRVal, nodeId)) = pr
        print('Page rank ', ind+1, 'for page id ',nodeId+1,'with value is - ', pRVal)

    print('\n\n')

In [7]:
def getBottomK(k_bottom,r):
    # Bottom 5 page ranks with values
    k_bottom_Pages_id = r.T[0].argsort()[:k_bottom]
    k_bottom_Page_list = []
    for ind in k_bottom_Pages_id:
        k_bottom_Page_list.append((r[ind][0], ind+1))
    
    # Printing bottom k nodes with scores
    print('----------------------Bottom k node ids with scores-----------------------')
    for pr in enumerate(k_bottom_Page_list):
        (ind, (pRVal, nodeId)) = pr
        print('Page rank ', ind+1, 'for page id ',nodeId+1,'with value is - ', pRVal)
    
    print('\n\n')

In [8]:
# Code to get graph data as graph edges.
edges_of_graph = dict()

with open('C:/Users/rvrma/Documents/3rd semester/numerical analysis/project/web-Stanford.txt') as file:
    line = file.readline()
    while line:
        line_list = []
        for node in line.strip().split('\t'):
            line_list.append(int(node))
        
        if edges_of_graph.get(line_list[0]):
            edges_of_graph[line_list[0]].append(line_list[1])
        else:
            edges_of_graph[line_list[0]] = [line_list[1]]
        
        line = file.readline()

#print(edges_of_graph)
tot_num_of_nodes = sorted(edges_of_graph.keys(), reverse=True)[:1][0]
print("Total number of nodes using are :- ", tot_num_of_nodes)

Total number of nodes using are :-  281903


In [9]:
# code to get sparse matrix from graph
row_list = []
col_list = []
data_list = []
for node in edges_of_graph.keys():
    node_vals = edges_of_graph.get(node)
    node_cnt = len(node_vals)
    each_n_val = 1 / node_cnt
    
    values_dict = dict()
    for value in node_vals:
        if values_dict.get(value):
            values_dict[value] = values_dict.get(value) + 1
        else:
            values_dict[value] = 1
    
    for n_val in list(set(node_vals)):
        count = values_dict.get(n_val)
        row_list.append(n_val-1)
        col_list.append(node-1)
        data_list.append(count * float(each_n_val))
    
Sparse_Matrix = sparse.csr_matrix((data_list, (row_list, col_list)), shape=(tot_num_of_nodes, tot_num_of_nodes))
num_of_non_zero_edges = sparse.csr_matrix.count_nonzero(Sparse_Matrix)
print('Total number of non zero points are :- ',num_of_non_zero_edges)
print("-----------------------Sparse Matrix------------------------")
print(Sparse_Matrix)

Total number of non zero points are :-  2312497
-----------------------Sparse Matrix------------------------
  (1, 871)	1.0
  (1, 1280)	1.0
  (1, 12093)	0.010869565217391304
  (1, 17092)	0.3333333333333333
  (1, 17793)	1.0
  (1, 19146)	0.5
  (1, 20922)	1.0
  (1, 25201)	1.0
  (1, 31701)	1.0
  (1, 35876)	1.0
  (1, 52410)	1.0
  (1, 53624)	1.0
  (1, 54581)	1.0
  (1, 59347)	1.0
  (1, 64929)	1.0
  (1, 72939)	0.034482758620689655
  (1, 73763)	1.0
  (1, 78294)	1.0
  (1, 84476)	1.0
  (1, 98007)	1.0
  (1, 98627)	1.0
  (1, 100192)	1.0
  (1, 102354)	1.0
  (1, 105317)	1.0
  (1, 105729)	0.3333333333333333
  :	:
  (281898, 30563)	0.037037037037037035
  (281898, 85038)	0.037037037037037035
  (281898, 125083)	0.037037037037037035
  (281898, 234925)	0.037037037037037035
  (281898, 266497)	0.3333333333333333
  (281898, 274183)	0.037037037037037035
  (281898, 275193)	0.3333333333333333
  (281899, 60320)	0.01
  (281899, 99268)	0.009900990099009901
  (281899, 151631)	0.009900990099009901
  (281899, 180112)	

In [10]:
# code to get weighted sparse matrix from graph
row_list = []
col_list = []
data_list = []

for node in edges_of_graph.keys():
    node_vals = edges_of_graph.get(node)
    node_cnt = len(node_vals)
    each_n_val = 1 / node_cnt
    
    values_dict = dict()
    for value in node_vals:
        if values_dict.get(value):
            values_dict[val] = values_dict.get(value) + 1
        else:
            values_dict[value] = 1
    for n_val in list(set(node_vals)):
        cnt = values_dict.get(n_val)
        data_list.append(cnt)
        row_list.append(n_val-1)
        col_list.append(node-1)
        
matrix = sparse.csr_matrix((data_list, (row_list, col_list)), shape=(tot_num_of_nodes, tot_num_of_nodes))

r_list = []
c_list = []
data_list = []
r_sum = matrix.sum(axis=1).T.tolist()[0]
c_sum = matrix.sum(axis=0).tolist()[0]

for node in edges_of_graph.keys():
    node_vals = edges_of_graph.get(node)
    
    n_sum_in = 0
    for n_val in list(set(node_vals)):
        n_sum_in = n_sum_in + r_sum[n_val-1]
        
    n_sum_out = 0
    for n_val in list(set(node_vals)):
        n_sum_out = n_sum_in + c_sum[n_val-1]
        
    for n_val in list(set(node_vals)):
        n_in_c = r_sum[n_val-1]
        w_in = float(n_in_c/n_sum_in)
        n_out_c = c_sum[n_val-1]
        w_out =float(n_out_c/n_sum_out)
        r_list.append(n_val-1)
        c_list.append(node-1)
        weight = w_in * w_out
        data_list.append(weight)
    
Weighted_Sparse_Matrix = sparse.csr_matrix((data_list, (r_list, c_list)), shape=(tot_num_of_nodes, tot_num_of_nodes))
print("-----------------------Weighted Sparse Matrix------------------------")
print(Weighted_Sparse_Matrix)

-----------------------Weighted Sparse Matrix------------------------
  (1, 871)	0.3069306930693069
  (1, 1280)	0.3069306930693069
  (1, 12093)	2.688071811404882e-06
  (1, 17092)	0.0731304552960604
  (1, 17793)	0.3069306930693069
  (1, 19146)	0.2552340625735121
  (1, 20922)	0.3069306930693069
  (1, 25201)	0.3069306930693069
  (1, 31701)	0.3069306930693069
  (1, 35876)	0.3069306930693069
  (1, 52410)	0.3069306930693069
  (1, 53624)	0.3069306930693069
  (1, 54581)	0.3069306930693069
  (1, 59347)	0.3069306930693069
  (1, 64929)	0.3069306930693069
  (1, 72939)	5.604950755367591e-07
  (1, 73763)	0.3069306930693069
  (1, 78294)	0.3069306930693069
  (1, 84476)	0.3069306930693069
  (1, 98007)	0.3069306930693069
  (1, 98627)	0.3069306930693069
  (1, 100192)	0.3069306930693069
  (1, 102354)	0.3069306930693069
  (1, 105317)	0.3069306930693069
  (1, 105729)	0.0731304552960604
  :	:
  (281898, 30563)	9.608609313945295e-05
  (281898, 85038)	9.608609313945295e-05
  (281898, 125083)	9.608609313945295e

In [21]:
# Using normal page rank and power iteration
r = power_iteration(Sparse_Matrix)
print("Normal page rank with power iteration :- ")
getTopK(5,r)
getBottomK(5,r)

For tolerance -  1e-10 Total number of iterations for power iteration it took is :-  86


Normal page rank with power iteration :- 
---------------------Top k node ids with scores------------------------
Page rank  1 for page id  89074 with value is -  0.010474629007599982
Page rank  2 for page id  226412 with value is -  0.009610237778224624
Page rank  3 for page id  241455 with value is -  0.008379659180575367
Page rank  4 for page id  134833 with value is -  0.0032578572592821694
Page rank  5 for page id  69359 with value is -  0.0027316738913733996



----------------------Bottom k node ids with scores-----------------------
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  21387 with value is -  7.094638936087944e-07
Page rank  3 for page id  87192 with value is -  7.094638936087944e-07
Page rank  4 for page id  205726 with value is -  7.094638936087944e-07
Page rank  5 for page id  205720 with value is -  7.094638936087944e-07





In [22]:
# Using normal page rank and power extrapolation
r = power_extrapolation(Sparse_Matrix, tot_num_of_nodes)
print("Normal page rank with power extrapolation :- ")
getTopK(5,r)
getBottomK(5,r)

Total number of iterations for power extrapolation it took is :-  75


Normal page rank with power extrapolation :- 
---------------------Top k node ids with scores------------------------
Page rank  1 for page id  89074 with value is -  0.010474629005360647
Page rank  2 for page id  226412 with value is -  0.009610237776892454
Page rank  3 for page id  241455 with value is -  0.00837965917897915
Page rank  4 for page id  134833 with value is -  0.003257857259166489
Page rank  5 for page id  69359 with value is -  0.002731673891306296



----------------------Bottom k node ids with scores-----------------------
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  21387 with value is -  7.094638936087944e-07
Page rank  3 for page id  87192 with value is -  7.094638936087944e-07
Page rank  4 for page id  205726 with value is -  7.094638936087944e-07
Page rank  5 for page id  205720 with value is -  7.094638936087944e-07





In [23]:
# Using weighted page rank and power iteration
r = power_iteration(Weighted_Sparse_Matrix)
print("weighted page rank with power iteration :- ")
getTopK(5,r)
getBottomK(5,r)

For tolerance -  1e-10 Total number of iterations for power iteration it took is :-  17


weighted page rank with power iteration :- 
---------------------Top k node ids with scores------------------------
Page rank  1 for page id  82869 with value is -  0.00010306948712871268
Page rank  2 for page id  160278 with value is -  7.603265457011555e-05
Page rank  3 for page id  27685 with value is -  7.224273255526352e-05
Page rank  4 for page id  25636 with value is -  6.72899123907457e-05
Page rank  5 for page id  20707 with value is -  6.672859648514436e-05



----------------------Bottom k node ids with scores-----------------------
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  185682 with value is -  7.094638936087944e-07
Page rank  3 for page id  185689 with value is -  7.094638936087944e-07
Page rank  4 for page id  185698 with value is -  7.094638936087944e-07
Page rank  5 for page id  185701 with value is -  7.094638936087944e-07





In [24]:
# Using weighted page rank and power extrapolation
r = power_extrapolation(Weighted_Sparse_Matrix,tot_num_of_nodes)
print("weighted page rank with power extrapolation :- ")
getTopK(5,r)
getBottomK(5,r)

Total number of iterations for power extrapolation it took is :-  24


weighted page rank with power extrapolation :- 
---------------------Top k node ids with scores------------------------
Page rank  1 for page id  82869 with value is -  0.00010306948712871268
Page rank  2 for page id  160278 with value is -  7.603265457011554e-05
Page rank  3 for page id  27685 with value is -  7.224273255526352e-05
Page rank  4 for page id  25636 with value is -  6.72899123907437e-05
Page rank  5 for page id  20707 with value is -  6.672859648514235e-05



----------------------Bottom k node ids with scores-----------------------
Page rank  1 for page id  2 with value is -  7.094638936087944e-07
Page rank  2 for page id  185682 with value is -  7.094638936087944e-07
Page rank  3 for page id  185689 with value is -  7.094638936087944e-07
Page rank  4 for page id  185698 with value is -  7.094638936087944e-07
Page rank  5 for page id  185701 with value is -  7.094638936087944e-07



